Data From UC Irvine Online Retail Data Repository

Data Preparation Based on DataCamp Customer Segmentation Video

In [81]:
import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


In [82]:
retail = pd.read_excel('Online Retail.xlsx')
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [83]:
retail.shape

(541909, 8)

In [84]:
retail.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [85]:
retail.CustomerID.nunique()

4372

Cohort Analysis


In [86]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [87]:
#Function to get year month and first day of the month from invoice date
def get_month(x): return dt.datetime(x.year, x.month, 1)

retail['InvoiceMonth'] = retail['InvoiceDate'].apply(get_month)
group = retail.groupby('CustomerID')['InvoiceMonth']
retail['CohortMonth'] = group.transform('min')
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,CohortMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01


In [88]:
#Function to retrieve date in integer format
def get_date_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day

In [89]:
invoice_year, invoice_month, _ = get_date_int(retail, 'InvoiceMonth')
cohort_year, cohort_month, _ = get_date_int(retail, 'CohortMonth')

years_diff = invoice_year - cohort_year
months_diff = invoice_month - cohort_month
#Cohort is the month in which customers made their first purchase. It begins at 1
#For the earliest cohort. Below transformation is done to get the cohort index.
retail['CohortIndex'] = years_diff * 12 + months_diff + 1
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,CohortMonth,CohortIndex
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,2010-12-01,1.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01,1.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,2010-12-01,1.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01,1.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01,1.0


In [90]:
#Get monthly active customers from each cohort
grouping = retail.groupby(['CohortMonth', 'CohortIndex'])

cohort_data = grouping['CustomerID'].apply(pd.Series.nunique)
cohort_data = cohort_data.reset_index()
cohort_counts = cohort_data.pivot(index='CohortMonth', 
                                columns='CohortIndex', 
                                values='CustomerID')


In [91]:
#Size of each starting cohort
cohort_sizes = cohort_counts.iloc[:,0]

#Calculate retention rate
retention = cohort_counts.divide(cohort_sizes, axis=0)
retention = retention.round(3) * 100

retention

CohortIndex,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0
CohortMonth,,,,,,,,,,,,,
2010-12-01,100.0,38.2,33.4,38.7,36.0,39.7,38.0,35.4,35.4,39.5,37.3,50.0,27.4
2011-01-01,100.0,24.0,28.3,24.2,32.8,29.9,26.1,25.7,31.1,34.7,36.8,15.0,NaN
2011-02-01,100.0,24.7,19.2,27.9,26.8,24.7,25.5,28.2,25.8,31.3,9.2,NaN,NaN
2011-03-01,100.0,19.1,25.5,21.8,23.2,17.7,26.4,23.9,28.9,8.9,NaN,NaN,NaN
2011-04-01,100.0,22.7,22.1,21.1,20.7,23.7,23.1,26.1,8.4,NaN,NaN,NaN,NaN
2011-05-01,100.0,23.7,17.2,17.2,21.5,24.4,26.5,10.4,NaN,NaN,NaN,NaN,NaN
2011-06-01,100.0,20.9,18.7,27.2,24.7,33.6,10.2,NaN,NaN,NaN,NaN,NaN,NaN
2011-07-01,100.0,20.9,20.4,23.0,27.2,11.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08-01,100.0,25.1,25.1,25.1,13.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
def df_to_plotly(df):
    return {'z': df.values.tolist(),
        'x': df.columns.tolist(),
        'y': df.index.tolist()}

In [93]:
z = retention.values.tolist()

In [94]:
fig = px.imshow(z, text_auto=True, color_continuous_scale='Viridis')
fig.update_layout(title='Retention Rate Heatmap',
                    xaxis_title='Cohort Index',
                    yaxis_title='Cohort Month',
                    yaxis_nticks=24,
                    xaxis_nticks=24)

fig.show()

In [95]:
print(min(retail.InvoiceDate))
print(max(retail.InvoiceDate))

2010-12-01 08:26:00
2011-12-09 12:50:00


In [96]:
snapshot_date = max(retail.InvoiceDate) + dt.timedelta(days=1)
snapshot_date

Timestamp('2011-12-10 12:50:00')

In [97]:
retail['TotalSum'] = retail['UnitPrice'] * retail['Quantity']

In [98]:
#Select where CustomerID = 12747
retail[retail['CustomerID'] == 12747]['TotalSum'].sum()


4196.009999999999

In [99]:
#Create Recency, Frequency, Monetary Value Quantiles
temp = retail.groupby('CustomerID').agg({'InvoiceDate': lambda x: 
                            (snapshot_date - x.max()).days,
                            'InvoiceNo': 'count',
                            'TotalSum': 'sum'
})



In [100]:
#Convert CustomerID to index
temp.reset_index(inplace=True)

In [101]:
#Rename columns
temp.rename(columns={'InvoiceDate': 'Recency',
                            'InvoiceNo': 'Frequency',
                            'TotalSum': 'MonetaryValue'}, inplace=True)
                            

In [102]:
temp

,CustomerID,Recency,Frequency,MonetaryValue
0,12346.0,326,2,0.00
1,12347.0,2,182,4310.00
2,12348.0,75,31,1797.24
3,12349.0,19,73,1757.55
4,12350.0,310,17,334.40
...,...,...,...,...
4367,18280.0,278,10,180.60
4368,18281.0,181,7,80.82
4369,18282.0,8,13,176.60
4370,18283.0,4,756,2094.88


In [103]:
#Select customer 12747
temp[temp['CustomerID'] == 12747]

,CustomerID,Recency,Frequency,MonetaryValue
329,12747.0,2,103,4196.01


In [104]:
#Create RFM Segments
#Create labels for Recency and Frequency
r_labels = range(4, 0, -1); f_labels = range(1, 5)
m_labels = range(1, 5)
f_labels = range(1,5)

#Assign these labels to 4 equal percentile groups
r_groups = pd.qcut(temp['Recency'], q=4, labels=r_labels)
f_groups = pd.qcut(temp['Frequency'], q=4, labels=f_labels)
m_groups = pd.qcut(temp['MonetaryValue'], q=4, labels=m_labels)

#Create new columns R, F and M
temp = temp.assign(R = r_groups.values, F = f_groups.values, M = m_groups.values)
temp.head()


,CustomerID,Recency,Frequency,MonetaryValue,R,F,M
0,12346.0,326,2,0.00,1,1,1
1,12347.0,2,182,4310.00,4,4,4
2,12348.0,75,31,1797.24,2,2,4
3,12349.0,19,73,1757.55,3,3,4
4,12350.0,310,17,334.40,1,1,2


In [105]:
#Plot R vs Recency
fig = px.scatter(temp, x='R', y='Recency')
fig.show()

In [106]:
#import suplots for plotly
from plotly.subplots import make_subplots
#Plot 3 by 1 subplots of R, F and M
fig = make_subplots(rows=3, cols=1)

fig.add_trace(go.Scatter(x=temp['R'], y=temp['Recency'], mode='markers'), row=1, col=1)
fig.add_trace(go.Scatter(x=temp['F'], y=temp['Frequency'], mode='markers'), row=2, col=1)
fig.add_trace(go.Scatter(x=temp['M'], y=temp['MonetaryValue'], mode='markers'), row=3, col=1)


In [107]:
#Plot RFM distribution
fig = px.scatter_3d(temp, x='Recency', y='Frequency', z='MonetaryValue',
                color='R', symbol='F', size='M',
                color_continuous_scale='Viridis',
                opacity=0.7, width=800, height=800)
fig.update_layout(title='RFM Distribution')

In [108]:
#Generate RFM Segment

def join_rfm(x): return str(x['R']) + str(x['F']) + str(x['M'])
temp['RFM_Segment'] = temp.apply(join_rfm, axis=1)

#Generate RFM Score
temp['RFM_Score'] = temp[['R','F','M']].sum(axis=1)

temp.head()

,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score
0,12346.0,326,2,0.00,1,1,1,1.01.01.0,3
1,12347.0,2,182,4310.00,4,4,4,4.04.04.0,12
2,12348.0,75,31,1797.24,2,2,4,2.02.04.0,8
3,12349.0,19,73,1757.55,3,3,4,3.03.04.0,10
4,12350.0,310,17,334.40,1,1,2,1.01.02.0,4


In [109]:
temp.groupby('RFM_Score').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']
}).round(1)

Recency Frequency MonetaryValue      
             mean      mean          mean count
RFM_Score                                      
3           264.8       7.8         109.1   392
4           174.5      13.9         227.1   391
5           153.0      21.2         346.8   517
6            94.3      28.5         491.8   468
7            78.8      39.7         724.2   447
8            62.7      57.0         974.7   467
9            44.2      79.0        1369.6   411
10           31.3     115.3        1894.0   440
11           20.5     193.9        3845.7   368
12            6.7     371.8        8850.7   471

In [110]:
#Group customers into 3 segements
def segment_me(df):
    if df['RFM_Score'] >= 9:
        return 'A'
    elif ((df['RFM_Score'] >= 5) and (df['RFM_Score'] < 9)):
        return 'B'
    else:
        return 'C'

temp['General_Segment'] = temp.apply(segment_me, axis=1)
temp.head()



,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score,General_Segment
0,12346.0,326,2,0.00,1,1,1,1.01.01.0,3,C
1,12347.0,2,182,4310.00,4,4,4,4.04.04.0,12,A
2,12348.0,75,31,1797.24,2,2,4,2.02.04.0,8,B
3,12349.0,19,73,1757.55,3,3,4,3.03.04.0,10,A
4,12350.0,310,17,334.40,1,1,2,1.01.02.0,4,C


In [111]:
temp.groupby('General_Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']
}).round(1)

Recency Frequency MonetaryValue      
                   mean      mean          mean count
General_Segment                                      
A                  25.2     195.1        4130.3  1690
B                  98.9      36.1         625.8  1899
C                 219.7      10.9         168.0   783

In [112]:
#Order General Segment by A > B > C
temp['General_Segment'] = pd.Categorical(temp['General_Segment'],
                                        categories=['A', 'B', 'C'])
temp.head()


,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score,General_Segment
0,12346.0,326,2,0.00,1,1,1,1.01.01.0,3,C
1,12347.0,2,182,4310.00,4,4,4,4.04.04.0,12,A
2,12348.0,75,31,1797.24,2,2,4,2.02.04.0,8,B
3,12349.0,19,73,1757.55,3,3,4,3.03.04.0,10,A
4,12350.0,310,17,334.40,1,1,2,1.01.02.0,4,C


K-Means Clustering
- Need to normalize values to ensure that one variable does not highly skew results


In [113]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,CohortMonth,CohortIndex,TotalSum
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,2010-12-01,1.0,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01,1.0,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,2010-12-01,1.0,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01,1.0,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01,1.0,20.34


In [80]:
temp.head()

,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score,General_Segment,clusters
0,12346.0,326,2,0.00,1,1,1,1.01.01.0,3,C,1
1,12347.0,2,182,4310.00,4,4,4,4.04.04.0,12,A,7
2,12348.0,75,31,1797.24,2,2,4,2.02.04.0,8,B,3
3,12349.0,19,73,1757.55,3,3,4,3.03.04.0,10,A,0
4,12350.0,310,17,334.40,1,1,2,1.01.02.0,4,C,1


In [144]:
#Graph General Segments
fig = px.scatter_3d(temp, x='Recency', y='Frequency', z='MonetaryValue',
                color='General_Segment', symbol='General_Segment',
                color_continuous_scale='Viridis',
                opacity=0.7, width=800, height=800)
fig.update_layout(title='RFM Distribution')
fig.show()

In [114]:
attributes = ['Recency', 'Frequency', 'MonetaryValue']
#Make subplots
fig = make_subplots(rows=1, cols=3)
#Widen figure
fig.update_layout(width=1000, height=500)

#Plot each attribute
for i, a in enumerate(attributes):
    fig.add_trace(go.Box(y=temp[a], name=a), row=1, col=i+1)
    #add title
    fig.update_layout(title='RFM Attributes Distribution')

fig.show()

In [142]:
#Remove Statistical outlieres using 0.95 IQR rule for visualization
Q1 = temp.quantile(0.05)[attributes]
Q3 = temp.quantile(0.95)[attributes]
IQR = Q3 - Q1

for att in attributes:
    no_outlier = temp[~((temp[att] < (Q1[att] - 1.5 * IQR[att])) |(temp[att] > (Q3[att] + 1.5 * IQR[att])))]
    print('Number of outliers for {}: {}'.format(att, temp.shape[0] - no_outlier.shape[0]))

no_outlier.describe()

fig = make_subplots(rows=3, cols=1, subplot_titles=attributes)

fig.update_layout(width=900, height=1000)

#Plot each attribute
for i, a in enumerate(attributes):
    fig.add_trace(go.Box(y=no_outlier[a], x=no_outlier['General_Segment'], name=a), row=i+1, col=1)
    #add title
    fig.update_layout(title='RFM Attributes Distribution by Cluster')

fig.show()

Number of outliers for Recency: 0
Number of outliers for Frequency: 29
Number of outliers for MonetaryValue: 59


Need to Standard Scale for Kmeans

In [115]:
#Standard Scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
temp_scaled = temp.copy()
scaler.fit(temp_scaled[attributes])
temp_scaled[attributes] = scaler.transform(temp_scaled[attributes])
temp_scaled.head()

,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score,General_Segment
0,12346.0,2.322023,-0.391720,-0.231001,1,1,1,1.01.01.0,3,C
1,12347.0,-0.893733,0.382657,0.293432,4,4,4,4.04.04.0,12,A
2,12348.0,-0.169196,-0.266959,-0.012316,2,2,4,2.02.04.0,8,B
3,12349.0,-0.725005,-0.086271,-0.017146,3,3,4,3.03.04.0,10,A
4,12350.0,2.163220,-0.327188,-0.190312,1,1,2,1.01.02.0,4,C


In [116]:
#K-means clustering for range of k
sse = {}
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, max_iter=50).fit(temp_scaled[attributes])
    temp_scaled["clusters"] = kmeans.labels_
    sse[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center

#Plot SSE for each k using plotly
fig = px.line(x=list(sse.keys()), y=list(sse.values()))
fig.update_layout(title='The Elbow Method', xaxis_title='k', yaxis_title='SSE')
fig.show()




Elbow method suggests k=3


In [134]:
#K-means clustering for k=3
kmeans = KMeans(n_clusters=3, max_iter=100, random_state=42).fit(temp_scaled[attributes])

#Assign cluster labels to copy of temp data
temp_k3 = temp.copy()
temp_k3['Cluster'] = kmeans.labels_
temp_k3.head()


,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score,General_Segment,Cluster
0,12346.0,326,2,0.00,1,1,1,1.01.01.0,3,C,1
1,12347.0,2,182,4310.00,4,4,4,4.04.04.0,12,A,0
2,12348.0,75,31,1797.24,2,2,4,2.02.04.0,8,B,0
3,12349.0,19,73,1757.55,3,3,4,3.03.04.0,10,A,0
4,12350.0,310,17,334.40,1,1,2,1.01.02.0,4,C,1


#describe each cluster
temp_k3.groupby('Cluster').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count', 'sum']
}).round(1)


In [123]:
temp_k3.groupby('Cluster').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']
}).round(1)

Recency Frequency MonetaryValue      
           mean      mean          mean count
Cluster                                      
0          40.3     104.9        1950.6  3264
1         247.2      27.7         464.2  1096
2           5.0    2845.6      118713.0    12

Observations:
- Cluster 1 has a much larger average monetary value, but only 250 members. The result is that the total cluster value is about that of cluster 0.
- Cluster 0 forms the largest number of customers and is in betweeen clusters 1 and 2 for frequency, mean spending, and recency.
- Cluster 2 Is the lowest performing cluster, with a mean rececy of 247.4. This cluster could be seasonal shoppers, the result of an ad campaign?

In [146]:
#Reorder clusteres
temp_k3['Cluster'] = temp_k3['Cluster'].map({2:2,0:1, 1:0})
temp_k3.groupby('Cluster').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count', 'sum']
}).round(1)

Recency Frequency MonetaryValue                 
           mean      mean          mean count        sum
Cluster                                                 
0         247.2      27.7         464.2  1096   508760.9
1          40.3     104.9        1950.6  3264  6366748.6
2           5.0    2845.6      118713.0    12  1424556.3

In [145]:
#Plot RFM distribution with selection option for clusters
fig = px.scatter_3d(temp_k3, x='Recency', y='Frequency', z='MonetaryValue',
                color='Cluster', symbol='Cluster',
                color_continuous_scale='Viridis',
                opacity=0.7, width=800, height=800)
fig.update_layout(title='RFM Distribution with Clusters')
fig.show()

In [152]:
temp_k3[temp_k3['Cluster'] == 2]['MonetaryValue'].sum() / temp_k3['MonetaryValue'].sum()

0.171631925809209

Compare above to previous 3d scatterplt. There is definitly a difference in separation. The third cluster here is mainly comprised of the extreme outliers, and there appears to be better separation along recency compared to the general segment grouping done earlier. Perhaps these 12 customers could be classified as vip, then the next silver, then bronze. They comprise almost 20% of income, although playing around with clustering this group could be expanded to make the classes a more balanced pyramid. 

In [131]:
#Cluster vs Attributes
#make subplots
fig = make_subplots(rows=3, cols=1)
fig.update_layout(width=1000, height=1200)

#Plot each attribute
for i, a in enumerate(attributes):
    fig.add_trace(go.Box(y=temp_k3[a], x=temp_k3['Cluster'], name=a), row=i+1, col=1)
    #add title
    fig.update_layout(title='RFM Attributes Distribution by Cluster')

fig.show()


In [ ]:
#Plot RFM distribution with selection option for cluster

In [ ]:
#Its hard to see the data due to outliers, so we will remove them
#Remove Statistical outlieres using 0.95 IQR for Recency
Q1 = temp_k3.quantile(0.05)[attributes]
Q3 = temp_k3.quantile(0.95)[attributes]
IQR = Q3 - Q1

for att in attributes:
    no_outlier = temp_k3[~((temp_k3[att] < (Q1[att] - 1.5 * IQR[att])) |(temp_k3[att] > (Q3[att] + 1.5 * IQR[att])))]
    print('Number of outliers for {}: {}'.format(att, temp_k3.shape[0] - no_outlier.shape[0]))

no_outlier.describe()

fig = make_subplots(rows=3, cols=1, subplot_titles=attributes)

fig.update_layout(width=900, height=1000)

#Plot each attribute
for i, a in enumerate(attributes):
    fig.add_trace(go.Box(y=no_outlier[a], x=no_outlier['Cluster'], name=a), row=i+1, col=1)
    #add title
    fig.update_layout(title='RFM Attributes Distribution by Cluster')

fig.show()

In [130]:
fig = px.scatter_matrix(temp_k3, dimensions=attributes, color='Cluster')
fig.show()